In [ ]:
# BiLSTM

#from random import random
#from numpy import array
#from numpy import cumsum
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
#from keras.layers import Bidirectional
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input, Dense

filename = 'asdf'

inputs = prep_features(filename)
tempos = blah


# inputs and outputs
nfft = 
nwin = 
a1 = Input(shape=(nwin,nfft/2 + 1))
a2 = Input(shape=(nwin,nfft/2 + 1))
a3 = Input(shape=(nwin,nfft/2 + 1))
a4 = Input(shape=(nwin,nfft/2 + 1))
a5 = Input(shape=(nwin,nfft/2 + 1))
a6 = Input(shape=(nwin,nfft/2 + 1))
alist = [a1,a2,a3,a4,a5,a6]
a = keras.layers.Concatenate(axis=-1)(alist)

def fork (model, n=2):
    forks = []
    for i in range(n):
        f = Sequential()
        f.add (model)
        forks.append(f)
    return forks

# fork the inputs
left, right = fork(a)

# LSTM left branch
left.add(LSTM(output_dim=hidden_units, init='uniform', inner_init='uniform',
               forget_bias_init='one', return_sequences=True, activation='tanh',
               inner_activation='sigmoid'))
left.add(LSTM(output_dim=hidden_units, init='uniform', inner_init='uniform',
               forget_bias_init='one', return_sequences=True, activation='tanh',
               inner_activation='sigmoid'))
left.add(LSTM(output_dim=hidden_units, init='uniform', inner_init='uniform',
               forget_bias_init='one', return_sequences=True, activation='tanh',
               inner_activation='sigmoid'))
# LSTM right branch
right.add(LSTM(output_dim=hidden_units, init='uniform', inner_init='uniform',
               forget_bias_init='one', return_sequences=True, activation='tanh',
               inner_activation='sigmoid', go_backwards=True))
right.add(LSTM(output_dim=hidden_units, init='uniform', inner_init='uniform',
               forget_bias_init='one', return_sequences=True, activation='tanh',
               inner_activation='sigmoid', go_backwards=True))
right.add(LSTM(output_dim=hidden_units, init='uniform', inner_init='uniform',
               forget_bias_init='one', return_sequences=True, activation='tanh',
               inner_activation='sigmoid', go_backwards=True))

# combine
output = Sequential()
output.add(Merge([left, right], mode='concat'))
# add dense at end - beat or no beat are the 2 units at end
output.add(TimeDistributed(Dense(units=2, activation='softmax')))

# stochastic gradient descent vs ADAM
adm = Adam(lr=0.0001, decay=1e-5, momentum=0.9, nesterov=True)

# create computational graph 
model = Model(inputs=alist, outputs=output)
model.compile(loss='categorical_crossentropy', optimizer=adm)

print("Train...")
model.fit(X_train, Y_train, batch_size=1, nb_epoch=nb_epoches, validation_data=(X_test, Y_test), verbose=1, show_accuracy=True)




In [ ]:
# 6 spectrograms

import librosa
from scipy.signal import medfilt

def prep_features(filename):
    
    x, fs = librosa.load(filename, sr=44100)

    # 3 melspectrograms
    s1 = librosa.core.stft(x, n_fft=1024, hop_length=441, win_length=1024, window='hamming', center=True, pad_mode='constant')
    ms1 = librosa.feature.melspectrogram(sr=44100, S=s1)
    s2 = librosa.core.stft(x, n_fft=2048, hop_length=441, win_length=2048, window='hamming', center=True, pad_mode='constant')
    ms2 = librosa.feature.melspectrogram(sr=44100, S=s2)
    s3 = librosa.core.stft(x, n_fft=4096, hop_length=441, win_length=4096, window='hamming', center=True, pad_mode='constant')
    ms3 = librosa.feature.melspectrogram(sr=44100, S=s3)

    # 3 median spectrograms
    # pad edges
    p1 = numpy.pad(ms1, ((0,0),(0,1024//2)), 'reflect')
    p2 = numpy.pad(ms2, ((0,0),(0,2048//2)), 'reflect')
    p3 = numpy.pad(ms3, ((0,0),(0,4096//2)), 'reflect')
    # cut off the non-causal median leftover from the end
    m1 = medfilt(p1, kernel_size=(1,1024//100))[:-1024/2]
    m2 = medfilt(p2, kernel_size=(1,2048//100))[:-2048/2]
    m3 = medfilt(p3, kernel_size=(1,4096//100))[:-4096/2]
    # half-wave rectify
    h1 = np.maximum(ms1-m1,0)
    h2 = np.maximum(ms2-m2,0)
    h3 = np.maximum(ms3-m3,0)

    return ms1, ms2, ms3, h1, h2, h3

In [ ]:
# CNN

from keras.layers import Conv1D
from keras.layers import Flatten

def cnn(x, num_steps, win_size=1024, weight_decay=0.01):

    a = Input(shape=(num_steps, win_size))
    filters = 256
    kernel_size = 128
    net = Conv1D(filters, kernel_size, strides=4, kernel_initializer='he_normal',
               kernel_regularizer=keras.regularizers.l2(weight_decay))(a)
    net = Conv1D(filters, kernel_size, strides=4, kernel_initializer='he_normal',
               kernel_regularizer=keras.regularizers.l2(weight_decay))(net)
    net = Conv1D(filters, kernel_size, strides=4, kernel_initializer='he_normal',
               kernel_regularizer=keras.regularizers.l2(weight_decay))(net)
    # remove channel dimension
    net = Flatten(data_format='channels_last')(net)
    
    net = Dense(units=128, activation='linear')

def fork (model, n=2):
    
    forks = []
    for i in range(n):
        f = Sequential()
        f.add (model)
        forks.append(f)
        
    return forks


def bilstm(inputs):

    # fork the inputs
    left, right = fork(inputs)

    # LSTM left branch
    left.add(LSTM(output_dim=hidden_units, init='uniform', inner_init='uniform',
                   forget_bias_init='one', return_sequences=True, activation='tanh',
                   inner_activation='sigmoid'))
    left.add(LSTM(output_dim=hidden_units, init='uniform', inner_init='uniform',
                   forget_bias_init='one', return_sequences=True, activation='tanh',
                   inner_activation='sigmoid'))
    left.add(LSTM(output_dim=hidden_units, init='uniform', inner_init='uniform',
                   forget_bias_init='one', return_sequences=True, activation='tanh',
                   inner_activation='sigmoid'))
    # LSTM right branch
    right.add(LSTM(output_dim=hidden_units, init='uniform', inner_init='uniform',
                   forget_bias_init='one', return_sequences=True, activation='tanh',
                   inner_activation='sigmoid', go_backwards=True))
    right.add(LSTM(output_dim=hidden_units, init='uniform', inner_init='uniform',
                   forget_bias_init='one', return_sequences=True, activation='tanh',
                   inner_activation='sigmoid', go_backwards=True))
    right.add(LSTM(output_dim=hidden_units, init='uniform', inner_init='uniform',
                   forget_bias_init='one', return_sequences=True, activation='tanh',
                   inner_activation='sigmoid', go_backwards=True))

    # combine
    output = Sequential()
    output.add(Merge([left, right], mode='concat'))
    # add dense at end - beat or no beat are the 2 units at end
    output.add(TimeDistributed(Dense(units=2, activation='softmax')))
    
    return output